In [15]:
# import libraries
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
import copy
from sklearn.model_selection import train_test_split


# for importing data
import os
import torchvision
from torchvision import datasets, transforms, models
from torchvision.utils import make_grid

import matplotlib.pyplot as plt
from IPython import display
display.set_matplotlib_formats('svg')

/var/folders/gx/pwy0byr15pq8m62vhfygrw8w0000gn/T/ipykernel_49491/2870729131.py:20: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  display.set_matplotlib_formats('svg')


## Basic CNN with Batchsize 32

In [16]:
# transformations
transform = transforms.Compose([
    transforms.Resize(100),
    transforms.CenterCrop(100),
    transforms.ToTensor(), # normalizes to range [0,1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]) # further normalization
])

In [17]:
root = '../Data/Fruit360/fruits-360_dataset/fruits-360/'

train_data = datasets.ImageFolder(os.path.join(root, 'Training'), transform=transform)
test_data = datasets.ImageFolder(os.path.join(root, 'Test'), transform=transform)

torch.manual_seed(42)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

class_names = train_data.classes


print(f'Training images available: {len(train_data)}')
print(f'Testing images available:  {len(test_data)}')
print(f'Class types available:  {len(class_names)}')

Training images available: 67692
Testing images available:  22688
Class types available:  131


In [4]:
class basicCNN(nn.Module):
    def __init__(self):
        super().__init__()
    
        # print toggle
        self.print = False


        ### -------------- feature map layers -------------- ###
        # first convolution layer
        self.conv1  = nn.Conv2d(3,32,3,stride=1)
        self.bnorm1 = nn.BatchNorm2d(32) # input the number of channels in this layer
      

        # second convolution layer
        self.conv2  = nn.Conv2d(32,32,3,stride=1)
        self.bnorm2 = nn.BatchNorm2d(32) # input the number of channels in this layer
      
        # third convolution layer
        self.conv3  = nn.Conv2d(32,64,3,stride=1)
        self.bnorm3 = nn.BatchNorm2d(64) # input the number of channels in this layer


        ### -------------- linear decision layers -------------- ###
        self.fc1 = nn.Linear(10*10*64,1024)
        self.fc2 = nn.Linear(1024,512)
        self.fc3 = nn.Linear(512,131)

        # toggle for printing out tensor sizes during forward prop
        self.print = False

    def forward(self,x):
        if self.print: print(f'Input: {list(x.shape)}')
      
        # first block: convolution -> maxpool -> batchnorm -> relu
        x = F.max_pool2d(self.conv1(x),2)
        x = F.relu(self.bnorm1(x))
            
        if self.print: print(f'First CPR block: {list(x.shape)}')

        # second block: convolution -> maxpool -> batchnorm -> relu
        x = F.max_pool2d(self.conv2(x),2)
        x = F.relu(self.bnorm2(x))
        if self.print: print(f'Second CPR block: {list(x.shape)}')

        # third block: convolution -> maxpool -> batchnorm -> relu
        x = F.max_pool2d(self.conv3(x),2)
        x = F.relu(self.bnorm3(x))
        if self.print: print(f'Third CPR block: {list(x.shape)}')

        # reshape for linear layer
        x = x.view(-1,10*10*64)
        if self.print: print(f'Vectorized: {list(x.shape)}')
      
        # linear layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        if self.print: print(f'Final output: {list(x.shape)}')

        return F.log_softmax(x, dim=1)
    
# create the model instance
cnn = basicCNN()

In [5]:
lossfun = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(cnn.parameters(),lr=0.01,momentum=.9)

In [6]:
import time
start_time = time.time()

numepochs = 10

# initialize losses
CtrainLoss = torch.zeros(numepochs)
CtestLoss  = torch.zeros(numepochs)
CtrainAcc  = torch.zeros(numepochs)
CtestAcc   = torch.zeros(numepochs)
Ctrain_correct = []
Ctest_correct = []


# loop over epochs
for epochi in range(numepochs):
    
    # loop over training data batches
    cnn.train() # switch to train mode
    batchLoss = []
    batchAcc  = []
    Ctrn_corr = 0
    
    for b, (X,y) in enumerate(train_loader):
        b += 1
        # forward pass and loss
        yHat = cnn(X)
        loss = lossfun(yHat,y)

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # loss and accuracy from this batch
        batchLoss.append(loss.item())
        batchAcc.append( torch.mean((torch.argmax(yHat,axis=1) == y).float()).item() )
        batch_corr = (torch.argmax(yHat,axis=1) == y).sum()
        Ctrn_corr += batch_corr 
        
        # Print interim results
        if b%300 == 0:
            print(f'epoch: {epochi:2}  batch: {b:4} loss: {loss.item():10.8f} train accuracy: {Ctrn_corr.item()*100/(32*b):7.3f}%')

      # end of batch loop...

    # and get average losses and accuracies across the batches
    CtrainLoss[epochi] = np.mean(batchLoss)
    CtrainAcc[epochi]  = 100*np.mean(batchAcc)

    
    #### test performance (here done in batches!)
    cnn.eval() # switch to test mode
    batchAcc  = []
    batchLoss = []
    Ctst_corr = 0
    
    for b,(X,y) in enumerate(test_loader):
        # forward pass and loss
        with torch.no_grad():
            yHat = cnn(X)
            loss = lossfun(yHat,y)
            Ctst_corr += (torch.argmax(yHat,axis=1) == y).sum()
            
        # loss and accuracy from this batch
        batchLoss.append(loss.item())
        batchAcc.append( torch.mean((torch.argmax(yHat,axis=1) == y).float()).item() )
    # end of batch loop...

    
    # and get average losses and accuracies across the batches
    CtestLoss[epochi] = np.mean(batchLoss)
    CtestAcc[epochi]  = 100*np.mean(batchAcc)

    # print out a status update
    print(f'Finished epoch {epochi+1}/{numepochs}. Test accuracy = {CtestAcc[epochi]:.2f}%')

print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

epoch:  0  batch:  300 loss: 0.16509847 train accuracy:  54.083%
epoch:  0  batch:  600 loss: 0.17801775 train accuracy:  72.516%
epoch:  0  batch:  900 loss: 0.05421299 train accuracy:  80.368%
epoch:  0  batch: 1200 loss: 0.05876272 train accuracy:  84.622%
epoch:  0  batch: 1500 loss: 0.02339854 train accuracy:  87.306%
epoch:  0  batch: 1800 loss: 0.00554814 train accuracy:  89.234%
epoch:  0  batch: 2100 loss: 0.00419619 train accuracy:  90.693%
Finished epoch 1/10. Test accuracy = 94.43%
epoch:  1  batch:  300 loss: 0.00639857 train accuracy:  99.500%
epoch:  1  batch:  600 loss: 0.00101474 train accuracy:  99.609%
epoch:  1  batch:  900 loss: 0.00057422 train accuracy:  99.653%
epoch:  1  batch: 1200 loss: 0.00212555 train accuracy:  99.703%
epoch:  1  batch: 1500 loss: 0.05344861 train accuracy:  99.717%
epoch:  1  batch: 1800 loss: 0.00234797 train accuracy:  99.729%
epoch:  1  batch: 2100 loss: 0.00011228 train accuracy:  99.756%
Finished epoch 2/10. Test accuracy = 98.45%
ep

## Basic CNN with Data Agumentation

In [14]:
# train transformations
transform_train = transforms.Compose([
    transforms.Resize(100),
    transforms.CenterCrop(100),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(hue=0.4),
    transforms.ColorJitter(contrast=0.5),
    transforms.ToTensor(), # normalizes to range [0,1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]) # further normalization
])

# test transformations
transform_test = transforms.Compose([
    transforms.Resize(100),
    transforms.CenterCrop(100),
    transforms.ToTensor(), # normalizes to range [0,1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]) # further normalization
])

In [18]:
root = '../Data/Fruit360/fruits-360_dataset/fruits-360/'

train_data = datasets.ImageFolder(os.path.join(root, 'Training'), transform=transform_train)
test_data = datasets.ImageFolder(os.path.join(root, 'Test'), transform=transform_test)

torch.manual_seed(42)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)
test_loader = DataLoader(test_data, batch_size=100, shuffle=True)

class_names = train_data.classes


print(f'Training images available: {len(train_data)}')
print(f'Testing images available:  {len(test_data)}')
print(f'Class types available:  {len(class_names)}')

Training images available: 67692
Testing images available:  22688
Class types available:  131


In [16]:
class basicCNN(nn.Module):
    def __init__(self):
        super().__init__()


        ### -------------- feature map layers -------------- ###
        # first convolution layer
        self.conv1  = nn.Conv2d(3,32,3,stride=1)
        self.bnorm1 = nn.BatchNorm2d(32) # input the number of channels in this layer
      

        # second convolution layer
        self.conv2  = nn.Conv2d(32,32,3,stride=1)
        self.bnorm2 = nn.BatchNorm2d(32) # input the number of channels in this layer
      
        # third convolution layer
        self.conv3  = nn.Conv2d(32,64,3,stride=1)
        self.bnorm3 = nn.BatchNorm2d(64) # input the number of channels in this layer


        ### -------------- linear decision layers -------------- ###
        self.fc1 = nn.Linear(10*10*64,1024)
        self.fc2 = nn.Linear(1024,512)
        self.fc3 = nn.Linear(512,131)
 

    def forward(self,x):
 
      
        # first block: convolution -> maxpool -> batchnorm -> relu
        x = F.max_pool2d(self.conv1(x),2)
        x = F.relu(self.bnorm1(x))

        # second block: convolution -> maxpool -> batchnorm -> relu
        x = F.max_pool2d(self.conv2(x),2)
        x = F.relu(self.bnorm2(x)) 

        # third block: convolution -> maxpool -> batchnorm -> relu
        x = F.max_pool2d(self.conv3(x),2)
        x = F.relu(self.bnorm3(x))
  
        # reshape for linear layer
        x = x.view(-1,10*10*64)
      
        # linear layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)


        return F.log_softmax(x, dim=1)
    
# create the model instance
cnn = basicCNN()
lossfun = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(cnn.parameters(),lr=0.01,momentum=.9)

In [17]:
import time
start_time = time.time()

numepochs = 10

# initialize losses
CtrainLoss = torch.zeros(numepochs)
CtestLoss  = torch.zeros(numepochs)
CtrainAcc  = torch.zeros(numepochs)
CtestAcc   = torch.zeros(numepochs)
Ctrain_correct = []
Ctest_correct = []


# loop over epochs
for epochi in range(numepochs):
    
    # loop over training data batches
    cnn.train() # switch to train mode
    batchLoss = []
    batchAcc  = []
    Ctrn_corr = 0
    
    for b, (X,y) in enumerate(train_loader):
        b += 1
        # forward pass and loss
        yHat = cnn(X)
        loss = lossfun(yHat,y)

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # loss and accuracy from this batch
        batchLoss.append(loss.item())
        batchAcc.append( torch.mean((torch.argmax(yHat,axis=1) == y).float()).item() )
        batch_corr = (torch.argmax(yHat,axis=1) == y).sum()
        Ctrn_corr += batch_corr 
        
        # Print interim results
        if b%300 == 0:
            print(f'epoch: {epochi:2}  batch: {b:4} loss: {loss.item():10.8f} train accuracy: {Ctrn_corr.item()*100/(100*b):7.3f}%')

      # end of batch loop...

    # and get average losses and accuracies across the batches
    CtrainLoss[epochi] = np.mean(batchLoss)
    CtrainAcc[epochi]  = 100*np.mean(batchAcc)

    
    #### test performance (here done in batches!)
    cnn.eval() # switch to test mode
    batchAcc  = []
    batchLoss = []
    Ctst_corr = 0
    
    for b,(X,y) in enumerate(test_loader):
        # forward pass and loss
        with torch.no_grad():
            yHat = cnn(X)
            loss = lossfun(yHat,y)
            Ctst_corr += (torch.argmax(yHat,axis=1) == y).sum()
            
        # loss and accuracy from this batch
        batchLoss.append(loss.item())
        batchAcc.append( torch.mean((torch.argmax(yHat,axis=1) == y).float()).item() )
    # end of batch loop...

    
    # and get average losses and accuracies across the batches
    CtestLoss[epochi] = np.mean(batchLoss)
    CtestAcc[epochi]  = 100*np.mean(batchAcc)

    # print out a status update
    print(f'Finished epoch {epochi+1}/{numepochs}. Test accuracy = {CtestAcc[epochi]:.2f}%')

print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

epoch:  0  batch:  300 loss: 0.71017039 train accuracy:  42.023%
epoch:  0  batch:  600 loss: 0.20436464 train accuracy:  64.802%
Finished epoch 1/10. Test accuracy = 85.65%
epoch:  1  batch:  300 loss: 0.07304914 train accuracy:  96.473%
epoch:  1  batch:  600 loss: 0.03413671 train accuracy:  97.183%
Finished epoch 2/10. Test accuracy = 85.69%
epoch:  2  batch:  300 loss: 0.01299843 train accuracy:  98.570%
epoch:  2  batch:  600 loss: 0.00482858 train accuracy:  98.893%
Finished epoch 3/10. Test accuracy = 94.57%
epoch:  3  batch:  300 loss: 0.02039841 train accuracy:  99.250%
epoch:  3  batch:  600 loss: 0.04747273 train accuracy:  99.277%
Finished epoch 4/10. Test accuracy = 92.64%
epoch:  4  batch:  300 loss: 0.01539907 train accuracy:  99.497%
epoch:  4  batch:  600 loss: 0.00806915 train accuracy:  99.582%
Finished epoch 5/10. Test accuracy = 94.56%
epoch:  5  batch:  300 loss: 0.00713794 train accuracy:  99.777%
epoch:  5  batch:  600 loss: 0.00366056 train accuracy:  99.740%


## VGG16 Improving

In [103]:
vggmodel2 = vgg
vggmodel2.load_state_dict(torch.load('FRUITS-VGG-Model.pt'))
vggmodel2.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [104]:
lossfun = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(vggmodel2.parameters(),lr=0.001,momentum=.9)

In [105]:
import time
start_time = time.time()


numepochs = 6

# initialize losses
VNtrainLoss = torch.zeros(numepochs)
VNtestLoss  = torch.zeros(numepochs)
VNtrainAcc  = torch.zeros(numepochs)
VNtestAcc   = torch.zeros(numepochs)
VNtrain_correct = []
VNtest_correct = []


# loop over epochs
for epochi in range(numepochs):

    
    # loop over training data batches
    vggmodel2.train() # switch to train mode
    batchLoss = []
    batchAcc  = []
    VNtrn_corr = 0
    
    for b, (X,y) in enumerate(train_loader):
        
        b += 1
        # forward pass and loss
        yHat = vggmodel2(X)
        loss = lossfun(yHat,y)

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # loss and accuracy from this batch
        batchLoss.append(loss.item())
        batchAcc.append( torch.mean((torch.argmax(yHat,axis=1) == y).float()).item() )
        batch_corr = (torch.argmax(yHat,axis=1) == y).sum()
        VNtrn_corr += batch_corr 
        
        # Print interim results
        if b%100 == 0:
            print(f'epoch: {epochi:2}  batch: {b:4} loss: {loss.item():10.8f} train accuracy: {VNtrn_corr.item()*100/(100*b):7.3f}%')

      # end of batch loop...

    # and get average losses and accuracies across the batches
    VNtrainLoss[epochi] = np.mean(batchLoss)
    VNtrainAcc[epochi]  = 100*np.mean(batchAcc)

    
    #### test performance (here done in batches!)
    vggmodel2.eval() # switch to test mode
    batchAcc  = []
    batchLoss = []
    VNtst_corr = 0
    
    for b,(X,y) in enumerate(test_loader):
        # forward pass and loss
        with torch.no_grad():
            yHat = vggmodel2(X)
            loss = lossfun(yHat,y)
            VNtst_corr += (torch.argmax(yHat,axis=1) == y).sum()
            
        # loss and accuracy from this batch
        batchLoss.append(loss.item())
        batchAcc.append( torch.mean((torch.argmax(yHat,axis=1) == y).float()).item() )
    # end of batch loop...

    
    # and get average losses and accuracies across the batches
    VNtestLoss[epochi] = np.mean(batchLoss)
    VNtestAcc[epochi]  = 100*np.mean(batchAcc)

    # print out a status update
    print(f'Finished epoch {epochi+1}/{numepochs}. Test accuracy = {VNtestAcc[epochi]:.2f}%')

print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

epoch:  0  batch:  100 loss: 0.03889561 train accuracy:  92.770%
epoch:  0  batch:  200 loss: 0.07846430 train accuracy:  95.405%
epoch:  0  batch:  300 loss: 0.01480018 train accuracy:  96.457%
epoch:  0  batch:  400 loss: 0.01963034 train accuracy:  97.052%
epoch:  0  batch:  500 loss: 0.00926822 train accuracy:  97.536%
epoch:  0  batch:  600 loss: 0.00087650 train accuracy:  97.892%
Finished epoch 1/6. Test accuracy = 98.13%
epoch:  1  batch:  100 loss: 0.00063328 train accuracy:  99.760%
epoch:  1  batch:  200 loss: 0.00115976 train accuracy:  99.745%
epoch:  1  batch:  300 loss: 0.00023939 train accuracy:  99.750%
epoch:  1  batch:  400 loss: 0.03106214 train accuracy:  99.725%
epoch:  1  batch:  500 loss: 0.06194613 train accuracy:  99.732%
epoch:  1  batch:  600 loss: 0.00112885 train accuracy:  99.752%
Finished epoch 2/6. Test accuracy = 98.62%
epoch:  2  batch:  100 loss: 0.02393621 train accuracy:  99.680%
epoch:  2  batch:  200 loss: 0.00058390 train accuracy:  99.805%
epoc

## ResNet 34 Improving

In [42]:
resnet34model2 = resnet
resnet34model2.load_state_dict(torch.load('FRUITS-RESNET34-Model.pt'))
resnet34model2.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [46]:
lossfun = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet34model2.parameters(),lr=0.001,momentum=.9)

In [47]:
import time
start_time = time.time()

numepochs = 6

# initialize losses
RNtrainLoss = torch.zeros(numepochs)
RNtestLoss  = torch.zeros(numepochs)
RNtrainAcc  = torch.zeros(numepochs)
RNtestAcc   = torch.zeros(numepochs)
RNtrain_correct = []
RNtest_correct = []


# loop over epochs
for epochi in range(numepochs):

    
    # loop over training data batches
    resnet34model2.train() # switch to train mode
    batchLoss = []
    batchAcc  = []
    RNtrn_corr = 0
    
    for b, (X,y) in enumerate(train_loader):
        
        b += 1
        # forward pass and loss
        yHat = resnet34model2(X)
        loss = lossfun(yHat,y)

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # loss and accuracy from this batch
        batchLoss.append(loss.item())
        batchAcc.append( torch.mean((torch.argmax(yHat,axis=1) == y).float()).item() )
        batch_corr = (torch.argmax(yHat,axis=1) == y).sum()
        RNtrn_corr += batch_corr 
        
        # Print interim results
        if b%100 == 0:
            print(f'epoch: {epochi:2}  batch: {b:4} loss: {loss.item():10.8f} train accuracy: {RNtrn_corr.item()*100/(100*b):7.3f}%')

      # end of batch loop...

    # and get average losses and accuracies across the batches
    RNtrainLoss[epochi] = np.mean(batchLoss)
    RNtrainAcc[epochi]  = 100*np.mean(batchAcc)

    
    #### test performance (here done in batches!)
    resnet34model2.eval() # switch to test mode
    batchAcc  = []
    batchLoss = []
    RNtst_corr = 0
    
    for b,(X,y) in enumerate(test_loader):
        # forward pass and loss
        with torch.no_grad():
            yHat = resnet34model2(X)
            loss = lossfun(yHat,y)
            RNtst_corr += (torch.argmax(yHat,axis=1) == y).sum()
            
        # loss and accuracy from this batch
        batchLoss.append(loss.item())
        batchAcc.append( torch.mean((torch.argmax(yHat,axis=1) == y).float()).item() )
    # end of batch loop...

    
    # and get average losses and accuracies across the batches
    RNtestLoss[epochi] = np.mean(batchLoss)
    RNtestAcc[epochi]  = 100*np.mean(batchAcc)

    # print out a status update
    print(f'Finished epoch {epochi+1}/{numepochs}. Test accuracy = {RNtestAcc[epochi]:.2f}%')

print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

epoch:  0  batch:  100 loss: 0.01247818 train accuracy:  99.990%
epoch:  0  batch:  200 loss: 0.00635256 train accuracy:  99.990%
epoch:  0  batch:  300 loss: 0.00821499 train accuracy:  99.983%
epoch:  0  batch:  400 loss: 0.00704362 train accuracy:  99.985%
epoch:  0  batch:  500 loss: 0.00869639 train accuracy:  99.986%
epoch:  0  batch:  600 loss: 0.01135406 train accuracy:  99.987%
Finished epoch 1/6. Test accuracy = 97.62%
epoch:  1  batch:  100 loss: 0.00493810 train accuracy:  99.990%
epoch:  1  batch:  200 loss: 0.00983767 train accuracy:  99.990%
epoch:  1  batch:  300 loss: 0.00622975 train accuracy:  99.987%
epoch:  1  batch:  400 loss: 0.00837342 train accuracy:  99.990%
epoch:  1  batch:  500 loss: 0.00714298 train accuracy:  99.988%
epoch:  1  batch:  600 loss: 0.01094252 train accuracy:  99.990%
Finished epoch 2/6. Test accuracy = 97.48%
epoch:  2  batch:  100 loss: 0.00972671 train accuracy: 100.000%
epoch:  2  batch:  200 loss: 0.00595572 train accuracy:  99.995%
epoc